    # Clase - Computación Distribuida

    ## Pyspark Hands-on 
    #### Marcelo Medel Vergara - Diplomado Data Engineer USACH

## Instalación de librerías necesarias

Si utilizas Anaconda para administrar ambientes de desarrollo la mejor vía para asegurar que funcionen correctamente las librerías es instalando directo desde Conda. Adicionalmente se instalan todas las librerías que necesita Spark.
- `conda create -n pyspark-DE`
- `conda activate pyspark-DE`
- `conda install -c conda-forge pyspark python=3.10`

De igual forma se puede instalar a través de PIP, pero sin asegurar funcionamiento correcto ni la instalación de dependencias. Adicionalmente será necesario tener instalado Pandas.
- `conda create -n pyspark-DE python=3.10`
- `conda activate pyspark-DE`
- `pip install pyspark`
- `pip install pandas`.

Para Google Colab:
- `!pip install pyspark`

## SparkSession

SparkSession es una clase en PySpark que existe desde la versión 2.0 (2016) que simplifica la forma de trabajar con Spark, tanto en las configuraciones como en la manipulación de datos estructurados. 


#### Funcionalidades principales de SparkSession:

1. **Configura Spark**: Para profundizar en las configuraciones posibles de Spark, visite https://spark.apache.org/docs/latest/configuration.html.
    - `SparkSession.builder.appName("some name").**config("some config key,value")**.getOrCreate()`
    - `spark.conf.get("some config key")`

2. **Crear DataFrames**: permite leer y escribir (Input/Output) diversas fuentes de datos y crear DataFrames para la manipulación de datos.
    - `spark.createDataFrame(data [, schema])`
    - `spark.read.json("path to some json")`

3. **Ejecutar SQL**: facilita la ejecución de consultas en SQL sobre los DataFrames.
    - `spark.sql("query to some view")`
    
4. **Gestiona contexto de Spark**: facilita la configuración y acceso a diferentes componentes y funcionalidades de Spark
    - `spark.sql.shuffle.partitions`
    - `spark.executor.memory`
    - `spark.catalog.listTables()`
    - `spark.catalog.listColumns("someTable")`
    - `spark.udf.register("someName", someUdf)`

In [1]:
from pyspark.sql import SparkSession

spark = (
        SparkSession.builder
        .appName("hands-on-pyspark")
        .config("spark.executor.memory","2g")
        .getOrCreate()
)

spark.sparkContext.setLogLevel("ERROR") #ALL, OFF, ERROR, DEBUG, INFO, WARN
spark.active()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/09 20:36:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
spark.conf.get("spark.executor.memory")

'2g'

In [3]:
spark.sparkContext.getConf().getAll()

[('spark.app.name', 'hands-on-pyspark'),
 ('spark.executor.memory', '2g'),
 ('spark.driver.extraJavaOptions',
  '-Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/jdk.internal.ref=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED -Djdk.reflect.useDirectMethodHandle=false'),
 ('spark.app.startTime', '17

## DataFrame en Spark

Un DataFrame es una estructura de datos bidimensional similar a cualquier tabla en una base de datos estructurada. Algunas de las características que tiene un DataFrame en PySpark son:
- **Distribuido**: Los datos están distribuidos en un clúster de nodos, lo que permite el procesamiento paralelo.

- **Inmutable**: Cada transformación produce un nuevo DataFrame.

- **SQL**: Permite operaciones tipo SQL y ofrece una interfaz similar a Pandas pero a gran escala.

- **Conexiones diversas**: Puede leer datos de múltiples fuentes (ej: JSON, CSV, Parquet, JDBC).

- **Optimización Automática**: Utiliza *Catalyst Optimizer* para optimizar automáticamente las consultas.

### Creación de un DataFrame

In [4]:
from datetime import date

data = [
    {
    "nombre":"Marcelo",
    "nacimiento": date(1987,10,7),
    "mail":"marcelo@gmail.com",
    "x":7990,
    "y":15.26,
    "active":True
    },
    {
    "nombre":"Andrea",
    "nacimiento": date(1994,6,1),
    "mail":"andrea@gmail.com",
    "x":15860,
    "y":-10.15,
    "active":False
    },{
    "nombre":"Juan",
    "nacimiento": date(2000,1,1),
    "mail":"juan@gmail.com",
    "x":8520,
    "y":26.15,
    "active":True
    },
]

In [8]:
df1 = spark.createDataFrame(data)

In [9]:
df1

DataFrame[active: boolean, mail: string, nacimiento: date, nombre: string, x: bigint, y: double]

In [10]:
df1.show()

+------+-----------------+----------+-------+-----+------+
|active|             mail|nacimiento| nombre|    x|     y|
+------+-----------------+----------+-------+-----+------+
|  true|marcelo@gmail.com|1987-10-07|Marcelo| 7990| 15.26|
| false| andrea@gmail.com|1994-06-01| Andrea|15860|-10.15|
|  true|   juan@gmail.com|2000-01-01|   Juan| 8520| 26.15|
+------+-----------------+----------+-------+-----+------+



In [11]:
df1.collect()

[Row(active=True, mail='marcelo@gmail.com', nacimiento=datetime.date(1987, 10, 7), nombre='Marcelo', x=7990, y=15.26),
 Row(active=False, mail='andrea@gmail.com', nacimiento=datetime.date(1994, 6, 1), nombre='Andrea', x=15860, y=-10.15),
 Row(active=True, mail='juan@gmail.com', nacimiento=datetime.date(2000, 1, 1), nombre='Juan', x=8520, y=26.15)]

In [12]:
from pyspark.sql import Row

df2 = spark.createDataFrame(
    [
        Row(nombre="Marcelo", nacimiento=date(1987,10,1), mail="marcelo@gmail.com", x=2356, y=15.69, active=True),
        Row(nombre="Andrea", nacimiento=date(2000,1,1), mail="andrea@gmail.com", x=5982, y=15.69, active=True),
        Row(nombre="Juan", nacimiento=date(1994,11,1), mail="juan@gmail.com", x=146584, y=-58.69, active=False),
    ]
)
df2.show()

+-------+----------+-----------------+------+------+------+
| nombre|nacimiento|             mail|     x|     y|active|
+-------+----------+-----------------+------+------+------+
|Marcelo|1987-10-01|marcelo@gmail.com|  2356| 15.69|  true|
| Andrea|2000-01-01| andrea@gmail.com|  5982| 15.69|  true|
|   Juan|1994-11-01|   juan@gmail.com|146584|-58.69| false|
+-------+----------+-----------------+------+------+------+



In [13]:
df1.printSchema()

root
 |-- active: boolean (nullable = true)
 |-- mail: string (nullable = true)
 |-- nacimiento: date (nullable = true)
 |-- nombre: string (nullable = true)
 |-- x: long (nullable = true)
 |-- y: double (nullable = true)



In [14]:
df2.printSchema()

root
 |-- nombre: string (nullable = true)
 |-- nacimiento: date (nullable = true)
 |-- mail: string (nullable = true)
 |-- x: long (nullable = true)
 |-- y: double (nullable = true)
 |-- active: boolean (nullable = true)



### Data Types

In [18]:
from pyspark.sql.types import (
    StructField, # un campo estructurado
    StringType, # texto
    DateType, # datetime.date
    IntegerType, # enteros
    FloatType, # 2.23
    BooleanType, # true/false
    StructType, # colección (array) de campos estructurados
    DecimalType, # DecimalType(10,2)
    ArrayType, # lista, tupla, ...
    LongType, 
    DoubleType,
    MapType
)

cols = [
    StructField("nombre", StringType(), False),
    StructField("nombre_completo", StructType([
        StructField("nombre", StringType(),False),
        StructField("apellido", StringType(),False),
    ]) ),
    StructField("nacimiento", DateType(), False),
    StructField("mail", StringType(), False),
    StructField("x", IntegerType(), False),
    StructField("y", FloatType(), False),
    StructField("active", BooleanType(), False),
]

schema = StructType(cols)

df3 = spark.createDataFrame([],schema)
df3.show()
df3.printSchema()

+------+---------------+----------+----+---+---+------+
|nombre|nombre_completo|nacimiento|mail|  x|  y|active|
+------+---------------+----------+----+---+---+------+
+------+---------------+----------+----+---+---+------+

root
 |-- nombre: string (nullable = false)
 |-- nombre_completo: struct (nullable = true)
 |    |-- nombre: string (nullable = false)
 |    |-- apellido: string (nullable = false)
 |-- nacimiento: date (nullable = false)
 |-- mail: string (nullable = false)
 |-- x: integer (nullable = false)
 |-- y: float (nullable = false)
 |-- active: boolean (nullable = false)



In [19]:
from datetime import date

data = [
    {
    "nombre":"Marcelo",
    "nombre_completo":["Marcelo","Medel"],
    "nacimiento": date(1987,10,7),
    "mail":"marcelo@gmail.com",
    "x":7990,
    "y":15.26,
    "active":True
    },
    {
    "nombre":"Andrea",
    "nombre_completo":["Andrea","Vergara"],
    "nacimiento": date(1994,6,1),
    "mail":"andrea@gmail.com",
    "x":15860,
    "y":-10.15,
    "active":False
    },{
    "nombre":"Juan",
    "nombre_completo":["Juan","Bautista"],
    "nacimiento": date(2000,1,1),
    "mail":"juan@gmail.com",
    "x":8520,
    "y":26.15,
    "active":True
    },
]

df = spark.createDataFrame(data, schema)
df.show()
df.printSchema()

+-------+-----------------+----------+-----------------+-----+------+------+
| nombre|  nombre_completo|nacimiento|             mail|    x|     y|active|
+-------+-----------------+----------+-----------------+-----+------+------+
|Marcelo| {Marcelo, Medel}|1987-10-07|marcelo@gmail.com| 7990| 15.26|  true|
| Andrea|{Andrea, Vergara}|1994-06-01| andrea@gmail.com|15860|-10.15| false|
|   Juan| {Juan, Bautista}|2000-01-01|   juan@gmail.com| 8520| 26.15|  true|
+-------+-----------------+----------+-----------------+-----+------+------+

root
 |-- nombre: string (nullable = false)
 |-- nombre_completo: struct (nullable = true)
 |    |-- nombre: string (nullable = false)
 |    |-- apellido: string (nullable = false)
 |-- nacimiento: date (nullable = false)
 |-- mail: string (nullable = false)
 |-- x: integer (nullable = false)
 |-- y: float (nullable = false)
 |-- active: boolean (nullable = false)



## Operaciones básicas sobre un DataFrame

### Select, Columns y Expressions

***Select*** permite seleccionar un set de columnas y también puede ser usado para renombrar o aplicar *expresiones* a las columnas.

***Columns*** en Spark hace referencia a cualquier columna conocida en algun RDBMS, planilla de excel, pandas dataframes, etc. Estas columnas pueden ser seleccionadas, manipuladas o removidas de un DataFrame, por lo que estas operaciones son representadas como ***expressions***

-  `col` y `column` son utilizadas para referencias a columnas en un DataFrame
- `expr` es utilizado para crear expresiones mas complejas, recibe un string con SQL para ejecutar la operación.


In [25]:
from pyspark.sql.functions import col, column, expr

df.select('x'
          ,df.nacimiento
          ,col("y")
          ,column("nombre_completo")
          ,expr("nombre_completo")
          ).show()

+-----+----------+------+-----------------+-----------------+
|    x|nacimiento|     y|  nombre_completo|  nombre_completo|
+-----+----------+------+-----------------+-----------------+
| 7990|1987-10-07| 15.26| {Marcelo, Medel}| {Marcelo, Medel}|
|15860|1994-06-01|-10.15|{Andrea, Vergara}|{Andrea, Vergara}|
| 8520|2000-01-01| 26.15| {Juan, Bautista}| {Juan, Bautista}|
+-----+----------+------+-----------------+-----------------+



In [26]:
df.select('nombre_completo.*'
          ,'nombre_completo.apellido'
          ,col('nombre_completo.apellido')
          ,expr('nombre_completo.nombre')
          ).show()

+-------+--------+--------+--------+-------+
| nombre|apellido|apellido|apellido| nombre|
+-------+--------+--------+--------+-------+
|Marcelo|   Medel|   Medel|   Medel|Marcelo|
| Andrea| Vergara| Vergara| Vergara| Andrea|
|   Juan|Bautista|Bautista|Bautista|   Juan|
+-------+--------+--------+--------+-------+



In [34]:
df.select(
    expr("*"),
    expr("round(x*1.19)+1500 as valor_iva"),
    expr("split(mail,'@') as asd")
).show()

+-------+-----------------+----------+-----------------+-----+------+------+---------+--------------------+
| nombre|  nombre_completo|nacimiento|             mail|    x|     y|active|valor_iva|                 asd|
+-------+-----------------+----------+-----------------+-----+------+------+---------+--------------------+
|Marcelo| {Marcelo, Medel}|1987-10-07|marcelo@gmail.com| 7990| 15.26|  true|    11008|[marcelo, gmail.com]|
| Andrea|{Andrea, Vergara}|1994-06-01| andrea@gmail.com|15860|-10.15| false|    20373| [andrea, gmail.com]|
|   Juan| {Juan, Bautista}|2000-01-01|   juan@gmail.com| 8520| 26.15|  true|    11639|   [juan, gmail.com]|
+-------+-----------------+----------+-----------------+-----+------+------+---------+--------------------+



In [37]:
df2 = df.select(
    expr("*"),
    expr("round(x*1.19)+1500 as valor_iva"),
    expr("split(mail,'@') as asd")
)

In [35]:
df.selectExpr("*","round(x*1.19)+1500 as valor_iva").show()

+-------+-----------------+----------+-----------------+-----+------+------+---------+
| nombre|  nombre_completo|nacimiento|             mail|    x|     y|active|valor_iva|
+-------+-----------------+----------+-----------------+-----+------+------+---------+
|Marcelo| {Marcelo, Medel}|1987-10-07|marcelo@gmail.com| 7990| 15.26|  true|    11008|
| Andrea|{Andrea, Vergara}|1994-06-01| andrea@gmail.com|15860|-10.15| false|    20373|
|   Juan| {Juan, Bautista}|2000-01-01|   juan@gmail.com| 8520| 26.15|  true|    11639|
+-------+-----------------+----------+-----------------+-----+------+------+---------+



In [44]:
df.selectExpr("*","round(x*1.19)+1500 as valor_iva").selectExpr("avg(valor_iva)", "max(valor_iva)", "min(valor_iva)").show()

+--------------+--------------+--------------+
|avg(valor_iva)|max(valor_iva)|min(valor_iva)|
+--------------+--------------+--------------+
|    14340.0000|         20373|         11008|
+--------------+--------------+--------------+



### Literals, withColumn, withColumnRenamed y alias

In [46]:
from pyspark.sql.functions import lit

df.select(expr("*"), lit(1).alias("flag")).show()

df.selectExpr("*", "1 as flag").show()

+-------+-----------------+----------+-----------------+-----+------+------+----+
| nombre|  nombre_completo|nacimiento|             mail|    x|     y|active|flag|
+-------+-----------------+----------+-----------------+-----+------+------+----+
|Marcelo| {Marcelo, Medel}|1987-10-07|marcelo@gmail.com| 7990| 15.26|  true|   1|
| Andrea|{Andrea, Vergara}|1994-06-01| andrea@gmail.com|15860|-10.15| false|   1|
|   Juan| {Juan, Bautista}|2000-01-01|   juan@gmail.com| 8520| 26.15|  true|   1|
+-------+-----------------+----------+-----------------+-----+------+------+----+

+-------+-----------------+----------+-----------------+-----+------+------+----+
| nombre|  nombre_completo|nacimiento|             mail|    x|     y|active|flag|
+-------+-----------------+----------+-----------------+-----+------+------+----+
|Marcelo| {Marcelo, Medel}|1987-10-07|marcelo@gmail.com| 7990| 15.26|  true|   1|
| Andrea|{Andrea, Vergara}|1994-06-01| andrea@gmail.com|15860|-10.15| false|   1|
|   Juan| {Juan

In [52]:
df3 = df.selectExpr("*", expr("round(x*1.19) as valor_iva"))
df3.show()

PySparkTypeError: [NOT_ITERABLE] Column is not iterable.

In [49]:
#df.withColumn("flag",lit(2)).show()
df2 = df.withColumn("valor_iva",expr("round(x*1.19)"))
df2.show()

+-------+-----------------+----------+-----------------+-----+------+------+---------+
| nombre|  nombre_completo|nacimiento|             mail|    x|     y|active|valor_iva|
+-------+-----------------+----------+-----------------+-----+------+------+---------+
|Marcelo| {Marcelo, Medel}|1987-10-07|marcelo@gmail.com| 7990| 15.26|  true|     9508|
| Andrea|{Andrea, Vergara}|1994-06-01| andrea@gmail.com|15860|-10.15| false|    18873|
|   Juan| {Juan, Bautista}|2000-01-01|   juan@gmail.com| 8520| 26.15|  true|    10139|
+-------+-----------------+----------+-----------------+-----+------+------+---------+



In [54]:
df2.withColumnRenamed("nombre_completo","nombre_2").select("nombre_2.nombre").show()
df2 = df2.withColumnRenamed("nombre_completo","nombre_2")
df2.show() 

+-------+
| nombre|
+-------+
|Marcelo|
| Andrea|
|   Juan|
+-------+

+-------+-----------------+----------+-----------------+-----+------+------+---------+
| nombre|         nombre_2|nacimiento|             mail|    x|     y|active|valor_iva|
+-------+-----------------+----------+-----------------+-----+------+------+---------+
|Marcelo| {Marcelo, Medel}|1987-10-07|marcelo@gmail.com| 7990| 15.26|  true|     9508|
| Andrea|{Andrea, Vergara}|1994-06-01| andrea@gmail.com|15860|-10.15| false|    18873|
|   Juan| {Juan, Bautista}|2000-01-01|   juan@gmail.com| 8520| 26.15|  true|    10139|
+-------+-----------------+----------+-----------------+-----+------+------+---------+



In [56]:
df2 = df2.withColumnsRenamed(
            {"nombre_completo":"nombre2",
                        "x":"pago",
                        "y":'uso'}
                       )
df2.show()

+-------+-----------------+----------+-----------------+-----+------+------+---------+
| nombre|         nombre_2|nacimiento|             mail| pago|   uso|active|valor_iva|
+-------+-----------------+----------+-----------------+-----+------+------+---------+
|Marcelo| {Marcelo, Medel}|1987-10-07|marcelo@gmail.com| 7990| 15.26|  true|     9508|
| Andrea|{Andrea, Vergara}|1994-06-01| andrea@gmail.com|15860|-10.15| false|    18873|
|   Juan| {Juan, Bautista}|2000-01-01|   juan@gmail.com| 8520| 26.15|  true|    10139|
+-------+-----------------+----------+-----------------+-----+------+------+---------+



### Filter, where, drop y cast

In [65]:
df.show()

+-------+-----------------+----------+-----------------+-----+------+------+
| nombre|  nombre_completo|nacimiento|             mail|    x|     y|active|
+-------+-----------------+----------+-----------------+-----+------+------+
|Marcelo| {Marcelo, Medel}|1987-10-07|marcelo@gmail.com| 7990| 15.26|  true|
| Andrea|{Andrea, Vergara}|1994-06-01| andrea@gmail.com|15860|-10.15| false|
|   Juan| {Juan, Bautista}|2000-01-01|   juan@gmail.com| 8520| 26.15|  true|
+-------+-----------------+----------+-----------------+-----+------+------+



In [67]:
#df.filter(df.active).show()
#df.filter("active").show()
#df.filter("active == True").show()
#df.filter("active == true").show()
#df.filter("active == 1").show()
#df.filter("active = 0").show()

#df.filter("x > 8000").show()
df.filter(df.x > 8000).show()


+------+-----------------+----------+----------------+-----+------+------+
|nombre|  nombre_completo|nacimiento|            mail|    x|     y|active|
+------+-----------------+----------+----------------+-----+------+------+
|Andrea|{Andrea, Vergara}|1994-06-01|andrea@gmail.com|15860|-10.15| false|
|  Juan| {Juan, Bautista}|2000-01-01|  juan@gmail.com| 8520| 26.15|  true|
+------+-----------------+----------+----------------+-----+------+------+



In [71]:
df.where("x > 8000").show()
df.selectExpr("*","round(x*1.19) as pago").where("pago > 10000").show()

+------+-----------------+----------+----------------+-----+------+------+
|nombre|  nombre_completo|nacimiento|            mail|    x|     y|active|
+------+-----------------+----------+----------------+-----+------+------+
|Andrea|{Andrea, Vergara}|1994-06-01|andrea@gmail.com|15860|-10.15| false|
|  Juan| {Juan, Bautista}|2000-01-01|  juan@gmail.com| 8520| 26.15|  true|
+------+-----------------+----------+----------------+-----+------+------+

+------+-----------------+----------+----------------+-----+------+------+-----+
|nombre|  nombre_completo|nacimiento|            mail|    x|     y|active| pago|
+------+-----------------+----------+----------------+-----+------+------+-----+
|Andrea|{Andrea, Vergara}|1994-06-01|andrea@gmail.com|15860|-10.15| false|18873|
|  Juan| {Juan, Bautista}|2000-01-01|  juan@gmail.com| 8520| 26.15|  true|10139|
+------+-----------------+----------+----------------+-----+------+------+-----+



In [74]:
df = df.withColumn("x_float", col("x").cast("float"))
df = df.withColumn("x_str", col("x").cast("string"))
df.show()

+-------+-----------------+----------+-----------------+-----+------+------+-------+-----+
| nombre|  nombre_completo|nacimiento|             mail|    x|     y|active|x_float|x_str|
+-------+-----------------+----------+-----------------+-----+------+------+-------+-----+
|Marcelo| {Marcelo, Medel}|1987-10-07|marcelo@gmail.com| 7990| 15.26|  true| 7990.0| 7990|
| Andrea|{Andrea, Vergara}|1994-06-01| andrea@gmail.com|15860|-10.15| false|15860.0|15860|
|   Juan| {Juan, Bautista}|2000-01-01|   juan@gmail.com| 8520| 26.15|  true| 8520.0| 8520|
+-------+-----------------+----------+-----------------+-----+------+------+-------+-----+



In [75]:
df.printSchema()

root
 |-- nombre: string (nullable = false)
 |-- nombre_completo: struct (nullable = true)
 |    |-- nombre: string (nullable = false)
 |    |-- apellido: string (nullable = false)
 |-- nacimiento: date (nullable = false)
 |-- mail: string (nullable = false)
 |-- x: integer (nullable = false)
 |-- y: float (nullable = false)
 |-- active: boolean (nullable = false)
 |-- x_float: float (nullable = false)
 |-- x_str: string (nullable = false)



### Pandas DataFrame to Spark DataFrame

![spark-pandas](./files/spark_pandas.png)

In [76]:
from datetime import date

data = [
    {
        "nombre_completo": ["Carla", "Pérez"],
        "nacimiento": date(1985, 2, 14),
        "mail": "carla.p@gmail.com",
        "x": 6012,
        "y": 20.54,
        "active": True
    },
    {
        "nombre_completo": ["Pedro", "González"],
        "nacimiento": date(1992, 11, 3),
        "mail": "pedro.g@gmail.com",
        "x": 4521,
        "y": 9.8,
        "active": False
    },
    {
        "nombre_completo": ["Lucía", "Ramírez"],
        "nacimiento": date(1989, 5, 22),
        "mail": "lucia.r@gmail.com",
        "x": 3790,
        "y": 12.34,
        "active": True
    },
    {
        "nombre_completo": ["Diego", "Martínez"],
        "nacimiento": date(1995, 12, 19),
        "mail": "diego.m@gmail.com",
        "x": 5120,
        "y": 6.87,
        "active": True
    },
    {
        "nombre_completo": ["Ana", "López"],
        "nacimiento": date(1987, 8, 30),
        "mail": "ana.l@gmail.com",
        "x": 4250,
        "y": 10.52,
        "active": False
    },
    {
        "nombre_completo": ["Fernando", "Sánchez"],
        "nacimiento": date(1991, 3, 17),
        "mail": "fernando.s@gmail.com",
        "x": 5789,
        "y": 13.67,
        "active": True
    },
    {
        "nombre_completo": ["Marta", "Rivas"],
        "nacimiento": date(1994, 6, 8),
        "mail": "marta.r@gmail.com",
        "x": 4892,
        "y": 8.9,
        "active": False
    },
    {
        "nombre_completo": ["Luis", "Ortiz"],
        "nacimiento": date(1988, 9, 29),
        "mail": "luis.o@gmail.com",
        "x": 3400,
        "y": 14.6,
        "active": True
    },
    {
        "nombre_completo": ["Elena", "Torres"],
        "nacimiento": date(1993, 4, 15),
        "mail": "elena.t@gmail.com",
        "x": 4603,
        "y": 7.45,
        "active": True
    },
    {
        "nombre_completo": ["Javier", "Fernández"],
        "nacimiento": date(1986, 7, 21),
        "mail": "javier.f@gmail.com",
        "x": 5950,
        "y": 18.7,
        "active": False
    },
    {
        "nombre_completo": ["Patricia", "Suárez"],
        "nacimiento": date(1990, 1, 11),
        "mail": "patricia.s@gmail.com",
        "x": 3290,
        "y": 10.85,
        "active": True
    },
    {
        "nombre_completo": ["Raúl", "Vega"],
        "nacimiento": date(1992, 10, 10),
        "mail": "raul.v@gmail.com",
        "x": 5125,
        "y": 12.2,
        "active": False
    },
    {
        "nombre_completo": ["Sofía", "Gutiérrez"],
        "nacimiento": date(1984, 5, 5),
        "mail": "sofia.g@gmail.com",
        "x": 4701,
        "y": 11.1,
        "active": True
    },
    {
        "nombre_completo": ["David", "Morales"],
        "nacimiento": date(1996, 12, 25),
        "mail": "david.m@gmail.com",
        "x": 4156,
        "y": 9.3,
        "active": True
    },
    {
        "nombre_completo": ["Claudia", "Navarro"],
        "nacimiento": date(1989, 7, 16),
        "mail": "claudia.n@gmail.com",
        "x": 3958,
        "y": 13.9,
        "active": False
    },
    {
        "nombre_completo": ["Pablo", "Reyes"],
        "nacimiento": date(1991, 11, 7),
        "mail": "pablo.r@gmail.com",
        "x": 5012,
        "y": 8.6,
        "active": True
    },
    {
        "nombre_completo": ["Adriana", "Castro"],
        "nacimiento": date(1985, 6, 27),
        "mail": "adriana.c@gmail.com",
        "x": 3700,
        "y": 11.75,
        "active": True
    },
    {
        "nombre_completo": ["Carlos", "Herrera"],
        "nacimiento": date(1993, 2, 12),
        "mail": "carlos.h@gmail.com",
        "x": 4520,
        "y": 7.9,
        "active": False
    },
    {
        "nombre_completo": ["María", "Molina"],
        "nacimiento": date(1995, 9, 1),
        "mail": "maria.m@gmail.com",
        "x": 4398,
        "y": 10.35,
        "active": True
    },
    {
        "nombre_completo": ["Gustavo", "Cruz"],
        "nacimiento": date(1988, 12, 18),
        "mail": "gustavo.c@gmail.com",
        "x": 5803,
        "y": 16.4,
        "active": False
    },
    {
        "nombre_completo": ["Natalia", "Blanco"],
        "nacimiento": date(1990, 3, 22),
        "mail": "natalia.b@gmail.com",
        "x": 4990,
        "y": 9.67,
        "active": True
    }
]

import pandas as pd

pdf = pd.DataFrame(data)
pdf.head()

,nombre_completo,nacimiento,mail,x,y,active
0,"[Carla, Pérez]",1985-02-14,carla.p@gmail.com,6012,20.54,True
1,"[Pedro, González]",1992-11-03,pedro.g@gmail.com,4521,9.80,False
2,"[Lucía, Ramírez]",1989-05-22,lucia.r@gmail.com,3790,12.34,True
3,"[Diego, Martínez]",1995-12-19,diego.m@gmail.com,5120,6.87,True
4,"[Ana, López]",1987-08-30,ana.l@gmail.com,4250,10.52,False


In [80]:
pdf.head().T

,0,1,2,3,4
nombre_completo,"[Carla, Pérez]","[Pedro, González]","[Lucía, Ramírez]","[Diego, Martínez]","[Ana, López]"
nacimiento,1985-02-14,1992-11-03,1989-05-22,1995-12-19,1987-08-30
mail,carla.p@gmail.com,pedro.g@gmail.com,lucia.r@gmail.com,diego.m@gmail.com,ana.l@gmail.com
x,6012,4521,3790,5120,4250
y,20.54,9.8,12.34,6.87,10.52
active,True,False,True,True,False


In [77]:
pdf

,nombre_completo,nacimiento,mail,x,y,active
0,"[Carla, Pérez]",1985-02-14,carla.p@gmail.com,6012,20.54,True
1,"[Pedro, González]",1992-11-03,pedro.g@gmail.com,4521,9.80,False
2,"[Lucía, Ramírez]",1989-05-22,lucia.r@gmail.com,3790,12.34,True
3,"[Diego, Martínez]",1995-12-19,diego.m@gmail.com,5120,6.87,True
4,"[Ana, López]",1987-08-30,ana.l@gmail.com,4250,10.52,False
5,"[Fernando, Sánchez]",1991-03-17,fernando.s@gmail.com,5789,13.67,True
6,"[Marta, Rivas]",1994-06-08,marta.r@gmail.com,4892,8.90,False
7,"[Luis, Ortiz]",1988-09-29,luis.o@gmail.com,3400,14.60,True
8,"[Elena, Torres]",1993-04-15,elena.t@gmail.com,4603,7.45,True
9,"[Javier, Fernández]",1986-07-21,javier.f@gmail.com,5950,18.70,False


In [78]:
df = spark.createDataFrame(pdf)
df.show()

+-------------------+----------+--------------------+----+-----+------+
|    nombre_completo|nacimiento|                mail|   x|    y|active|
+-------------------+----------+--------------------+----+-----+------+
|     [Carla, Pérez]|1985-02-14|   carla.p@gmail.com|6012|20.54|  true|
|  [Pedro, González]|1992-11-03|   pedro.g@gmail.com|4521|  9.8| false|
|   [Lucía, Ramírez]|1989-05-22|   lucia.r@gmail.com|3790|12.34|  true|
|  [Diego, Martínez]|1995-12-19|   diego.m@gmail.com|5120| 6.87|  true|
|       [Ana, López]|1987-08-30|     ana.l@gmail.com|4250|10.52| false|
|[Fernando, Sánchez]|1991-03-17|fernando.s@gmail.com|5789|13.67|  true|
|     [Marta, Rivas]|1994-06-08|   marta.r@gmail.com|4892|  8.9| false|
|      [Luis, Ortiz]|1988-09-29|    luis.o@gmail.com|3400| 14.6|  true|
|    [Elena, Torres]|1993-04-15|   elena.t@gmail.com|4603| 7.45|  true|
|[Javier, Fernández]|1986-07-21|  javier.f@gmail.com|5950| 18.7| false|
| [Patricia, Suárez]|1990-01-11|patricia.s@gmail.com|3290|10.85|

In [79]:
df.show(10, vertical=True)

-RECORD 0-------------------------------
 nombre_completo | [Carla, Pérez]       
 nacimiento      | 1985-02-14           
 mail            | carla.p@gmail.com    
 x               | 6012                 
 y               | 20.54                
 active          | true                 
-RECORD 1-------------------------------
 nombre_completo | [Pedro, González]    
 nacimiento      | 1992-11-03           
 mail            | pedro.g@gmail.com    
 x               | 4521                 
 y               | 9.8                  
 active          | false                
-RECORD 2-------------------------------
 nombre_completo | [Lucía, Ramírez]     
 nacimiento      | 1989-05-22           
 mail            | lucia.r@gmail.com    
 x               | 3790                 
 y               | 12.34                
 active          | true                 
-RECORD 3-------------------------------
 nombre_completo | [Diego, Martínez]    
 nacimiento      | 1995-12-19           
 mail           

In [81]:
pdf2 = df.toPandas()
pdf.head()

,nombre_completo,nacimiento,mail,x,y,active
0,"[Carla, Pérez]",1985-02-14,carla.p@gmail.com,6012,20.54,True
1,"[Pedro, González]",1992-11-03,pedro.g@gmail.com,4521,9.80,False
2,"[Lucía, Ramírez]",1989-05-22,lucia.r@gmail.com,3790,12.34,True
3,"[Diego, Martínez]",1995-12-19,diego.m@gmail.com,5120,6.87,True
4,"[Ana, López]",1987-08-30,ana.l@gmail.com,4250,10.52,False
